In [1]:
import numpy as np
import pickle

In [2]:
acts_path = '../data/cifar_r50_train/class_acts_test.pkl'

with open(acts_path, 'rb') as f:
    acts = pickle.load(f)
    

In [7]:
acts[0].shape

(1000, 2048)

In [13]:
np.var(acts[0])

0.007480971

In [41]:
class_acts_mean = np.mean(acts[2],axis=0)
cenetered_acts = acts[2] - class_acts_mean

class_cov = np.dot(cenetered_acts.T,cenetered_acts)/cenetered_acts.shape[0]

np.trace(class_cov)

4.179928

In [38]:
np.array([[1,3],[2,4], [2,4]]) - np.array([1,2])

array([[0, 1],
       [1, 2],
       [1, 2]])